In [ ]:
import h5py, sys, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
%matplotlib tk

In [ ]:
sys.path.append('/mnt/cbis/home/benedikt/.local/dragonfly/utils/py_src')

In [ ]:
import writeemc
import detector
import reademc

In [ ]:
path_to_data = "../../data/sparse/" # scratch/sparse on Maxwell
path_to_aux = "../../data/aux/" # scratch/benedikt/aux on Maxwell
path_to_geometry = "../../geometry/"
path_to_recons = "../../data/recons/"

## Reading metadata

In [ ]:
with h5py.File(path_to_aux + "sparse_hits_metadata.h5", "r") as f:
    cells = f['cells'][:]

## Reading 2D EMC classes (round 1)

In [ ]:
r = 1

In [ ]:
emc_folder = path_to_recons + "melv_0001/"
emc_output = emc_folder + "data/r%d/output_040.h5" %r
emc_photons = emc_folder + 'photons.txt'
emc_detector = path_to_recons + "det/det_lowq8.h5"

In [ ]:
photons_list = list(np.loadtxt(emc_photons, dtype=str))
photons_list = [emc_folder + p for p in photons_list]

In [ ]:
det = detector.Detector(emc_detector, mask_flag=True)
emc = reademc.EMCReader(photons_list, det)

In [ ]:
with h5py.File(emc_output,'r') as f:
    occupancies = f['occupancies'][:]
    likelihood = f['likelihood'][:]
    orientations = f['orientations'][:]
    images = f['intens'][:]
    print(list(f))

In [ ]:
nr_rot = 180
nr_static = 1
nr_modes = occupancies.shape[1] - nr_static
modes = orientations % (nr_modes)
rotind = orientations // (nr_modes)
modes[rotind >= nr_rot] = orientations[rotind >= nr_rot] - nr_modes * (nr_rot - 1)
ndata = len(modes)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(20,12), dpi=200)
for i in range(15):
    c,r = i//5, i%5
    axes[c,r].axis('off')
    nclass = (modes==i).sum()
    axes[c,r].set_title("%d/%d [%.1f%%]" %(nclass,ndata, nclass/ndata*100.))
    axes[c,r].imshow(images[i], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
plt.savefig("../../plots/2dclasses.png", bbox_inches='tight')
plt.show()

## Static class

In [ ]:
plt.figure(figsize=(5,5), dpi=100)
plt.axis('off')
plt.imshow(images[-1], norm=colors.LogNorm(vmin=0.0001, vmax=3), cmap='cividis')
plt.show()

## Exclude ring features and static features

In [ ]:
remove = 1,10,15

In [ ]:
blacklist = np.zeros(occupancies.shape[0], dtype=int)
for m in remove:
    blacklist[np.where(modes == m)[0]] = 1
ndata = blacklist.shape[0] - blacklist.sum()

In [ ]:
np.savetxt(emc_folder + "blacklist_%d_%d.dat" %(r,ndata), blacklist, fmt='%d')

## Select for ring features only

In [ ]:
remove = list(range(nr_modes+1))
remove.remove(1)
remove.remove(10)

In [ ]:
blacklist = np.zeros(occupancies.shape[0], dtype=int)
for m in remove:
    blacklist[np.where(modes == m)[0]] = 1
ndata = blacklist.shape[0] - blacklist.sum()

In [ ]:
np.savetxt(emc_folder + "blacklist_%d_%d.dat" %(r,ndata), blacklist, fmt='%d')